
1. **Extracción y empaquetado del pipeline de inferencia**

   * Se unificaron en `inference_pipeline.py` todas las piezas necesarias:

     * La carga del modelo EfficientNet y su transform de validación para inferir la condición tumoral.
     * Las funciones de preprocesado de la nota clínica y agregación de síntomas (reutilizando el código de `src/data_preprocessing.py`).
     * El cargado del preprocesador de características y del modelo XGBoost ya entrenados, junto con los codificadores de etiqueta.
     * Una única función `predict_treatment(...)` que recibe ruta de imagen, edad, sexo y texto clínico, y devuelve código y etiqueta tanto de condición como de tratamiento, junto a las probabilidades de cada clase.

2. **Construcción del microservicio con FastAPI**

   * Se creó `api/main.py`, que añade al `PYTHONPATH` las carpetas del proyecto y de código fuente (`src/`).
   * Se definió un endpoint POST `/predict` que acepta un archivo de imagen (`jpg`/`png`) y los campos de formulario (`age`, `sex`, `clinical_note`).
   * Dentro del endpoint:

     * Validamos el MIME type de la imagen.
     * Guardamos temporalmente el fichero recibido para pasar su ruta a la función de inferencia.
     * Invocamos `predict_treatment` y devolvemos su salida en formato JSON, limpiando el archivo temporal después.

3. **Ejecución del servidor**

   * Instalamos y usamos la misma versión de Python (3.12) en la que entrenamos el modelo para arrancar Uvicorn:

     ```bash
     py -3.12 -m uvicorn api.main:app --reload --host 127.0.0.1 --port 8000
     ```
   * Comprobamos en consola que Uvicorn levanta correctamente el servicio en `http://127.0.0.1:8000`.



In [ ]:
import requests

url = "http://127.0.0.1:8000/predict"

files = {
    "image": (
        "brain_glioma_0001.jpg",
        open(r"C:\Users\juanreur\Downloads\Reto Tecnico Biociencias\brain-cancer-ai-pipeline\data\Brain_Cancer raw MRI data\brain_cancer_IMG\brain_glioma\brain_glioma_0001.jpg", "rb"),
        "image/jpeg",
    )
}

data = {
    "age": 70,
    "sex": "F",
    "clinical_note": "Patient is experiencing memory lapses and speech difficulties"
}

resp = requests.post(url, files=files, data=data)
print(resp.status_code)
print(resp.json())


200
{'condition_code': 0, 'condition_label': 'Brain Glioma', 'treatment_code': 3, 'treatment_label': 'surgery', 'probabilities': [0.25187328457832336, 0.002291254233568907, 0.36525100469589233, 0.3805845081806183]}


## ¡ Prueba Exitosa !